In [1]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

from PIL import Image
import torchvision.transforms as transforms
import os
import numpy as np


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [4]:
batch_size = 1 # default:4
guidance_scale = 16.0 #default: 16.0

prompt = "a chair"
# prompt = "an office space with a large window that lets in natural light, a desk with a lamp for task lighting, and a lounge area with dimmable wall sconces for relaxed ambiance"
# prompt = "A modern, S-shape black chair with a minimalist design a circular base, featuring neon blue accent lighting along its contours"
# prompt = "a chair"
# prompt = "a circular wooden pavilion with a stone floor"
# prompt = "a fierce dragon with a feather wing"
# prompt = "a corgi"

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [5]:
# render_mode = 'nerf' # you can change this to 'stf' / 'nerf' (Neural Radiance Fields)
# size = 128 # this is the size of the renders; higher values take longer to render.


# to_pil = transforms.ToPILImage()
# save_dir = '../img'
# os.makedirs(save_dir, exist_ok=True)
# print('creating cameras...')
# cameras = create_pan_cameras(size, device)


# for i, latent in enumerate(latents):
#     print(f'creating images...{len(latents)-i}left')

#     images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
#     print(f'images done, saving...{len(latents)-i}left')
#     print('saving...')
#     # save
#     pil_images = []
#     for j, img in enumerate(images):
#         if isinstance(img, Image.Image):
#             # If img is already a PIL Image, no need to convert
#             pil_img = img
#         elif hasattr(img, 'cpu') and callable(getattr(img, 'cpu')):
#             # If it has a 'cpu' attribute, it might be a PyTorch tensor
#             pil_img = Image.fromarray(img.cpu().detach().numpy().astype('uint8'))
#         elif isinstance(img, np.ndarray):
#             # If it's a NumPy array, convert directly to PIL
#             pil_img = Image.fromarray(img)
#         else:
#             raise TypeError(f"Unsupported image type: {type(img)}")

#         # Save the image
#         pil_images.append(pil_img)
#         print(f'{j} saved')
    
#     gif_path = os.path.join(save_dir, f"{prompt}_{i}.gif")
#     pil_images[0].save(
#         gif_path,
#         save_all=True,
#         append_images=pil_images[1:],
#         duration=100,  # Duration between frames in milliseconds (adjust as needed)
#         loop=0  # Number of times the GIF should loop (0 means infinite)
#     )

#     display(gif_widget(images))



In [6]:
######## Example of saving the latents as meshes.
# import os
# from shap_e.util.notebooks import decode_latent_mesh

# folder_path = '../meshes'
# if not os.path.exists(folder_path):
#     os.makedirs(folder_path)


# filename_length = 10
# file_name = prompt[:filename_length]

# for i, latent in enumerate(latents):
#     t = decode_latent_mesh(xm, latent).tri_mesh()

#     ply_filename = os.path.join(folder_path, f'{file_name}_{i}.ply')
#     obj_filename = os.path.join(folder_path, f'{file_name}_{i}.obj')
#     with open(ply_filename, 'wb') as f:
#         t.write_ply(f)
#     # with open(obj_filename, 'w') as f:
#     #     t.write_obj(f)

In [15]:
######test api.py###############    
import os
from shap_e.util.notebooks import decode_latent_mesh

filename_length = 10
file_name = prompt[:filename_length]


# Define a path in a local directory where you have write permissions
# local_directory = '../../../data'


# if not os.path.exists(local_directory):
#         print(f'Creating directory {local_directory}')
#         os.makedirs(local_directory)

for i, latent in enumerate(latents):
        print(f'Generating mesh {i}')
        t = decode_latent_mesh(xm, latent).tri_mesh()
        
        print(f'Writing file {i}')
        # Construct the full file path
        # ply_filename = os.path.join(local_directory, f'{file_name}_{i}.ply')
        
        # with open(ply_filename, 'wb') as f:
        #         t.write_ply(f)

        local_directory = os.path.abspath('data')
        if not os.path.exists(local_directory):
                print(f'Creating directory {local_directory}')
                os.makedirs(local_directory)
        file_name = f'{file_name}_{i}.obj'
        full_path = os.path.join(local_directory, file_name)

        # obj_filename = os.path.join(local_directory, f'{file_name}_{i}.obj')
        with open(full_path, 'w') as f:
                t.write_obj(f)
        print(f'File {i} written')
        # Now send the file to the client
        # return send_file(ply_filename, as_attachment=True, download_name=f'{file_name}_{i}.ply')

Generating mesh 0
Writing file 0
File 0 written
